# Video Processing Notebook Guide
This notebook serves the purpose of video processing using various computational techniques.

## Importing Libraries
Here, we import essential libraries:
- `cv2`: OpenCV for image and video processing
- `mediapipe`: Google"s MediaPipe for pose estimation
- `os`: For operating system related tasks
- `pandas`: For DataFrame support

In [ ]:
import cv2
import mediapipe as mp
import os
import pandas as pd

## Setting Parameters
In this section, you can modify the following parameters:
- `MODEL`: Choose between Lite model (`1`) and Full model (`2`). Lite Model (`1`) is the  `Default`.
- `video_path`: Path to the video file.

In [ ]:
MODEL = 2  # 1 = Lite model, 2 = Full model
video_path = "C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/VIDEO_FILES/5012_I.MOV"  # Add your video file path here

if os.path.exists(video_path) == True:
    print(f"{video_path} is a valid file. Proceed with processing.")

else:
    raise ValueError(f"{video_path} does not exist. Try adding the entire file path.")

## Initialization
This part initializes MediaPipe components used in the notebook.

In [ ]:
# Initialize MediaPipe components
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

## Processing Loop
The core logic of video processing is performed in this loop.

In [ ]:
print(f"Processing video at {video_path}")
with mp_holistic.Holistic(static_image_mode=False, model_complexity=MODEL) as holistic:
    # Initialize DataFrame to store data
    data = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, image = cap.read()
        
        if not ret:
            print("Ignoring empty camera frame.")
            break

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)
        # Append data to list
        time_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
        if results.pose_landmarks is not None:
            right_wrist_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x
            right_wrist_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y
            data.append([time_ms, right_wrist_x, right_wrist_y])    

        # Video read error
        if not cap.isOpened():
            print(f"Error reading video file {video_path}, try adding full directory to path")

        # print(f"Processed frame at {time_ms}ms")
    cap.release()

In [ ]:
# Convert to DataFrame
df = pd.DataFrame(data, columns=["time_ms", "right_wrist_x", "right_wrist_y"])
df

## Data Output
Finally, the processed data is stored in a DataFrame and saved as a pickle file and csv file.

In [ ]:
# Print DataFrame shape
print(f"DataFrame Head: {df.head()}")

# # Save DataFrame as pickle file
# pickle_file_name = "C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/MOTION_TRACKING_FILES/" + os.path.splitext(os.path.basename(video_path))[0] + "_keypoints.pkl"
# df.to_pickle(pickle_file_name)
# print(f"DataFrame saved as {pickle_file_name}")

# Save DataFrame as CSV file
csv_file_name = "C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/MOTION_TRACKING_FILES/" + os.path.splitext(os.path.basename(video_path))[0] + "_keypoints.csv"
df.to_csv(csv_file_name, index=False)
print(f"DataFrame saved as {csv_file_name}")